## Antes de ejecutar
Ejecutar en el ambiente donde se esta trabajando, en nuestro caso Anaconda PowerShell Prompt el siguiente comando:
pip install selenium

Utilizaremos la siguiente URL para realizar webscrapping : 'quotes.toscrape.com'

In [227]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup

In [228]:
#Configuramos el navegador que utilizaremos, en nuestro caso usaremos Edge 
driver = webdriver.Edge()
# Chrome  = .Chrome
# Firefox = .Firefox
# Safari = .Safari

In [229]:
driver.get("https://quotes.toscrape.com/login") # Cargamos nuestra página
#Esta pestaña mostrará una ventana de login, en la cual podemos probar la funcion de escritura
time.sleep(2)

In [230]:
#Buscaremos el elemento llamado username, el cual espera una entrada de texto, le escribiremos admin con la siguiente linea de codigo
driver.find_element(By.NAME, "username").send_keys("admin")
#Lo mismo para Password
driver.find_element(By.NAME, "password").send_keys("admin")
time.sleep(2)

In [231]:
#Ahora buscara un boton que permite subir las entradas de texto que escribimos anteriormente, el submit, nos permitirá avanzar, le dará clic con la siguiente instruccion
driver.find_element(By.CSS_SELECTOR, 'input[type="submit"]').click()
time.sleep(2)

In [232]:
#Observaremos que ya estamos en la pagina de inicio loggeados, utilizaremos las cookies de autenticacion
cookies = driver.get_cookies()
session = requests.Session()  #Crea una sesión HTTP 
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

In [233]:
# utilizaremos una peticion GET para obtener el contenido de la pagina
response = session.get("https://quotes.toscrape.com/page/2/")

In [235]:
base_url = "https://quotes.toscrape.com"
next_url = "/page/1/"  # Empezamos desde la página 1

while next_url:
    print(f"Pagina: {base_url + next_url}")
    
    #  petición GET
    response = requests.get(base_url + next_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extraer las frases usando BeautifulSoup
    quotes = soup.find_all("div", class_="quote")
    print("\n--- Nueva página ---")
    
    for quote in quotes:
        text = quote.find("span", class_="text").text
        author = quote.find("small", class_="author").text
        tags = ", ".join([tag.text for tag in quote.find_all("a", class_="tag")])
        
        print(f"\n{text}\n{author}\n{tags}")
    
    # Buscar enlace a la siguiente página con el boton next
    next_li = soup.find("li", class_="next")
    time.sleep(2)
    if next_li:
        next_url = next_li.a["href"] 
        next_button = driver.find_element(By.XPATH, '//li[@class="next"]/a') # Al inicio se utilizo la funcion de buscar un boton por un css_selector, tambien podemos usar una busqueda por XPATH
        next_button.click()
    else:
        next_url = None  # En caso de que ya no haya más páginas, terminará la lectura
        print("No hay más páginas.")

Pagina: https://quotes.toscrape.com/page/1/

--- Nueva página ---

📜 “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
👤 Albert Einstein
🏷️ change, deep-thoughts, thinking, world

📜 “It is our choices, Harry, that show what we truly are, far more than our abilities.”
👤 J.K. Rowling
🏷️ abilities, choices

📜 “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
👤 Albert Einstein
🏷️ inspirational, life, live, miracle, miracles

📜 “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
👤 Jane Austen
🏷️ aliteracy, books, classic, humor

📜 “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
👤 Marilyn Monroe
🏷️ be-yourself, inspirational

📜 “Try not to become a man of success. Rather become a man of value.”
👤 Albert Einstein
🏷️ adulthood, succe

In [236]:
driver.quit() #Cierra el navegador